# Perocube Logbook Data Upload Notebook

This notebook processes and uploads logbook data from the Perocube Excel file to the TimescaleDB database.

## Purpose
- Read logbook data from 'PeroCube_logbook_example.xlsx'
- Parse and validate the data
- Upload the data to the TimescaleDB database
- Avoid duplicate data entries

## Prerequisites
- Running TimescaleDB instance (configured in docker-compose.yml)
- Access to the Perocube logbook Excel file
- Environment variables configured in .env file (for database connection)

## 1. Setup and Imports

Import required libraries and install any missing dependencies.

In [ ]:
# Core data processing libraries
import os
import pandas as pd
from datetime import datetime, timezone
from pathlib import Path

# Database libraries
from sqlalchemy import create_engine, text

# Progress tracking
from tqdm.notebook import tqdm

# Environment variables
from dotenv import load_dotenv

# Logging
import logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Install required packages if not already installed
!pip install psycopg2-binary sqlalchemy pandas tqdm pathlib python-dotenv openpyxl
import psycopg2

## 2. Configuration

Load configuration from environment variables and set up constants.

In [ ]:
# Look for the .env file two directories up from the notebook location
dotenv_path = Path("../../.env")
load_dotenv(dotenv_path)

# Database configuration from environment variables with fallbacks
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': int(os.getenv('DB_PORT', 5432)),
    'database': os.getenv('DB_NAME', 'perocube'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

# Print database connection info (excluding password)
print(f"Database connection: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']} as {DB_CONFIG['user']}")

# Data directory and file configuration
ROOT_DIRECTORY = os.getenv('DEFAULT_DATA_DIR', "../../sample_data/datasets/PeroCube-sample-data")
LOGBOOK_FILE = "PeroCube_logbook_example.xlsx"
LOGBOOK_SHEET = "Perocube history"

# Batch size for database operations
BATCH_SIZE = 1000

## 3. Read and Process Logbook Data

In [ ]:
# Create the full path to the logbook file
logbook_path = Path(ROOT_DIRECTORY) / LOGBOOK_FILE

# Read the Excel sheet, skip first row and use second row as header
try:
    df = pd.read_excel(logbook_path, sheet_name=LOGBOOK_SHEET, header=1)
    print(f"Successfully read {len(df)} rows from {LOGBOOK_FILE}")
    
    # Display the first few rows and data info
    print("\nColumn names:")
    print(df.columns.tolist())
    
    print("\nFirst few rows of the data:")
    display(df.head())
    
    print("\nDataset information:")
    display(df.info())
    
except Exception as e:
    print(f"Error reading Excel file: {str(e)}")

In [ ]:
# Analyze current dataframe state
print("Checking for unnamed columns:")
unnamed_cols = [col for col in df.columns if 'Unnamed' in str(col)]
print(f"Unnamed columns found: {unnamed_cols}")

print("\nCurrent data types:")
print(df.dtypes)

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nTotal rows with all missing values:")
print(df.isna().all(axis=1).sum())

In [ ]:
# Clean the dataframe

# 1. Remove unnamed columns
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

# 2. Drop rows where all values are missing
df = df.dropna(how='all')

# Print cleaning results
print("Dataframe shape after cleaning:")
print(f"Initial shape: {df.shape}")

# Display updated column list
print("\nUpdated column names:")
print(df.columns.tolist())

# Display updated missing values count
print("\nMissing values per column after cleaning:")
print(df.isnull().sum())

# Display first few rows of cleaned data
print("\nFirst few rows of cleaned data:")
display(df.head())

In [ ]:
# Remove completely empty Comment 2 column
df = df.drop('Comment 2', axis=1)

# Print updated dataframe info
print("Dataframe shape after removing Comment 2:")
print(f"Shape: {df.shape}")

# Display updated column list
print("\nUpdated column names:")
print(df.columns.tolist())

# Display first few rows of cleaned data
print("\nFirst few rows of cleaned data:")
display(df.head())